<a href="https://colab.research.google.com/github/eoinleen/protein-design-final-dir/blob/main/Ub2_binder_BSA_analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!/usr/bin/env python

import sys
import os
import numpy as np
import csv
from collections import defaultdict
from google.colab import drive
import glob

"""
Script to calculate interface contacts between a binder and K11-linked diubiquitin.
This is a distance-based approach that doesn't require external tools like FreeSASA.

For K11-linked Ub2 where the two ubiquitin units are combined into a single chain,
this script will:
1. Split the specified ubiquitin chain into two parts: Ub1 (residues 1-75) and Ub2 (residues 126-201)
2. Count the number of atoms within a contact distance threshold between the binder and each Ub unit
3. Calculate the contact surface area (approximate BSA) and the percentage per Ub unit

This script can process multiple PDB files in a directory and supports Google Drive integration.
"""

# Default settings - these can be changed directly here
DEFAULT_INPUT_DIR = "/content/drive/MyDrive/BindCraft/UB2-K11/3NOB/Att4-iso/Accepted"
DEFAULT_UB_CHAIN = "A"
DEFAULT_BINDER_CHAIN = "B"
DEFAULT_CUTOFF = 4.5
DEFAULT_OUTPUT_FILE = "results.csv"  # Will be saved in the input directory

def parse_pdb(pdb_file, ub_chain, binder_chain):
    """Parse PDB file and extract atom coordinates."""
    atoms = {
        'ub1': [],  # Ubiquitin chain, residues 1-75
        'ub2': [],  # Ubiquitin chain, residues 126-201
        'binder': [] # Binder chain
    }

    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM'):
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                chain = line[21:22]
                residue_num = int(line[22:26])
                atom_name = line[12:16].strip()
                residue_name = line[17:20].strip()

                if chain == ub_chain and 1 <= residue_num <= 75:
                    atoms['ub1'].append({
                        'coords': np.array([x, y, z]),
                        'residue_num': residue_num,
                        'atom_name': atom_name,
                        'residue_name': residue_name
                    })
                elif chain == ub_chain and 126 <= residue_num <= 201:
                    atoms['ub2'].append({
                        'coords': np.array([x, y, z]),
                        'residue_num': residue_num,
                        'atom_name': atom_name,
                        'residue_name': residue_name
                    })
                elif chain == binder_chain:
                    atoms['binder'].append({
                        'coords': np.array([x, y, z]),
                        'residue_num': residue_num,
                        'atom_name': atom_name,
                        'residue_name': residue_name
                    })

    return atoms

def calculate_contacts(atoms1, atoms2, cutoff=4.5):
    """Calculate contacts between two sets of atoms within a distance cutoff."""
    contacts = []

    for atom1 in atoms1:
        for atom2 in atoms2:
            dist = np.linalg.norm(atom1['coords'] - atom2['coords'])
            if dist < cutoff:
                contacts.append({
                    'atom1': atom1,
                    'atom2': atom2,
                    'distance': dist
                })

    return contacts

def calculate_interface_stats(contacts):
    """Calculate interface statistics from contacts."""
    if not contacts:
        return {
            'contact_count': 0,
            'residue_pairs': 0,
            'avg_distance': 0,
            'unique_residues1': 0,
            'unique_residues2': 0
        }

    # Extract unique residue pairs
    residue_pairs = set()
    unique_res1 = set()
    unique_res2 = set()

    for contact in contacts:
        res1 = contact['atom1']['residue_num']
        res2 = contact['atom2']['residue_num']
        residue_pairs.add((res1, res2))
        unique_res1.add(res1)
        unique_res2.add(res2)

    # Calculate average distance
    avg_distance = sum(contact['distance'] for contact in contacts) / len(contacts)

    # Return statistics
    return {
        'contact_count': len(contacts),
        'residue_pairs': len(residue_pairs),
        'avg_distance': avg_distance,
        'unique_residues1': len(unique_res1),
        'unique_residues2': len(unique_res2)
    }

def estimate_bsa(contact_count, residue_pairs):
    """Estimate BSA based on number of contacts and residue pairs."""
    # This is a rough approximation based on the correlation between
    # contact counts and actual BSA
    return contact_count * 0.5 + residue_pairs * 5

def calculate_per_residue_contacts(contacts):
    """Calculate how many contacts each residue is involved in."""
    res1_contacts = defaultdict(int)
    res2_contacts = defaultdict(int)

    for contact in contacts:
        res1 = contact['atom1']['residue_num']
        res2 = contact['atom2']['residue_num']
        res1_contacts[res1] += 1
        res2_contacts[res2] += 1

    return res1_contacts, res2_contacts

def get_top_residues(residue_contacts, n=5):
    """Get the top N residues with the most contacts."""
    sorted_contacts = sorted(residue_contacts.items(), key=lambda x: x[1], reverse=True)
    return sorted_contacts[:n]

def process_pdb_file(pdb_file, ub_chain, binder_chain, cutoff=4.5):
    """Process a single PDB file and return results as a dictionary."""
    # Parse PDB file
    atoms = parse_pdb(pdb_file, ub_chain, binder_chain)

    # Check if chains were found
    if not atoms['ub1'] or not atoms['ub2'] or not atoms['binder']:
        missing = []
        if not atoms['ub1']: missing.append(f"Ub1 (chain {ub_chain}, res 1-75)")
        if not atoms['ub2']: missing.append(f"Ub2 (chain {ub_chain}, res 126-201)")
        if not atoms['binder']: missing.append(f"Binder (chain {binder_chain})")

        print(f"Warning: Missing expected atoms in {pdb_file}: {', '.join(missing)}")

    # Calculate contacts
    ub1_binder_contacts = calculate_contacts(atoms['ub1'], atoms['binder'], cutoff)
    ub2_binder_contacts = calculate_contacts(atoms['ub2'], atoms['binder'], cutoff)

    # Calculate interface statistics
    ub1_stats = calculate_interface_stats(ub1_binder_contacts)
    ub2_stats = calculate_interface_stats(ub2_binder_contacts)

    # Estimate BSA
    ub1_bsa = estimate_bsa(ub1_stats['contact_count'], ub1_stats['residue_pairs'])
    ub2_bsa = estimate_bsa(ub2_stats['contact_count'], ub2_stats['residue_pairs'])
    total_bsa = ub1_bsa + ub2_bsa

    # Calculate percentages
    percentage_ub1 = (ub1_bsa / total_bsa) * 100 if total_bsa > 0 else 0
    percentage_ub2 = (ub2_bsa / total_bsa) * 100 if total_bsa > 0 else 0

    # Calculate per-residue contacts
    ub1_res_contacts, binder_ub1_contacts = calculate_per_residue_contacts(ub1_binder_contacts)
    ub2_res_contacts, binder_ub2_contacts = calculate_per_residue_contacts(ub2_binder_contacts)

    # Find top contact residues
    top_ub1_residues = get_top_residues(ub1_res_contacts)
    top_ub2_residues = get_top_residues(ub2_res_contacts)
    top_binder_residues = get_top_residues({**binder_ub1_contacts, **binder_ub2_contacts})

    # Determine binding uniformity
    diff = abs(percentage_ub1 - percentage_ub2)
    if diff < 10:
        uniformity = "Uniform"
    elif diff < 30:
        uniformity = "Moderate preference"
    else:
        uniformity = "Strong preference"

    # Format top residues as strings
    top_ub1_str = "; ".join([f"Res{res}:{count}" for res, count in top_ub1_residues])
    top_ub2_str = "; ".join([f"Res{res}:{count}" for res, count in top_ub2_residues])
    top_binder_str = "; ".join([f"Res{res}:{count}" for res, count in top_binder_residues])

    # Create result dictionary
    result = {
        "pdb_file": os.path.basename(pdb_file),
        "ub1_contacts": ub1_stats['contact_count'],
        "ub2_contacts": ub2_stats['contact_count'],
        "ub1_residue_pairs": ub1_stats['residue_pairs'],
        "ub2_residue_pairs": ub2_stats['residue_pairs'],
        "ub1_unique_residues": ub1_stats['unique_residues1'],
        "ub2_unique_residues": ub2_stats['unique_residues1'],
        "binder_ub1_unique_residues": ub1_stats['unique_residues2'],
        "binder_ub2_unique_residues": ub2_stats['unique_residues2'],
        "ub1_avg_distance": round(ub1_stats['avg_distance'], 2) if ub1_stats['avg_distance'] else 0,
        "ub2_avg_distance": round(ub2_stats['avg_distance'], 2) if ub2_stats['avg_distance'] else 0,
        "ub1_bsa": round(ub1_bsa, 2),
        "ub2_bsa": round(ub2_bsa, 2),
        "total_bsa": round(total_bsa, 2),
        "ub1_percentage": round(percentage_ub1, 2),
        "ub2_percentage": round(percentage_ub2, 2),
        "binding_uniformity": uniformity,
        "top_ub1_residues": top_ub1_str,
        "top_ub2_residues": top_ub2_str,
        "top_binder_residues": top_binder_str
    }

    return result

def print_result(result):
    """Print results for a single PDB file in a formatted way."""
    print("\n" + "="*70)
    print("INTERFACE ANALYSIS FOR K11-LINKED UBIQUITIN BINDER")
    print("="*70)
    print(f"Input PDB file: {result['pdb_file']}")

    print("\nContact Statistics:")
    print(f"  First Ubiquitin (residues 1-75):")
    print(f"    Atom contacts: {result['ub1_contacts']}")
    print(f"    Residue pairs: {result['ub1_residue_pairs']}")
    print(f"    Unique Ub1 residues involved: {result['ub1_unique_residues']}")
    print(f"    Unique Binder residues involved: {result['binder_ub1_unique_residues']}")
    print(f"    Average contact distance: {result['ub1_avg_distance']} Å")

    print(f"\n  Second Ubiquitin (residues 126-201):")
    print(f"    Atom contacts: {result['ub2_contacts']}")
    print(f"    Residue pairs: {result['ub2_residue_pairs']}")
    print(f"    Unique Ub2 residues involved: {result['ub2_unique_residues']}")
    print(f"    Unique Binder residues involved: {result['binder_ub2_unique_residues']}")
    print(f"    Average contact distance: {result['ub2_avg_distance']} Å")

    print("\nEstimated Buried Surface Areas (Å²):")
    print(f"  First Ubiquitin-Binder Interface:  {result['ub1_bsa']}")
    print(f"  Second Ubiquitin-Binder Interface: {result['ub2_bsa']}")
    print(f"  Total Estimated BSA:               {result['total_bsa']}")

    print("\nPercentage of Total Interface:")
    print(f"  First Ubiquitin:  {result['ub1_percentage']}%")
    print(f"  Second Ubiquitin: {result['ub2_percentage']}%")

    print("\nTop Contact Residues:")
    print(f"  First Ubiquitin: {result['top_ub1_residues']}")
    print(f"  Second Ubiquitin: {result['top_ub2_residues']}")
    print(f"  Binder: {result['top_binder_residues']}")

    print("\nBinding Uniformity Assessment:")
    print(f"  {result['binding_uniformity']}")

    print("="*70)

def run_analysis(input_dir=DEFAULT_INPUT_DIR,
                output_file=DEFAULT_OUTPUT_FILE,
                ub_chain=DEFAULT_UB_CHAIN,
                binder_chain=DEFAULT_BINDER_CHAIN,
                cutoff=DEFAULT_CUTOFF):
    """Run the analysis with the given parameters."""

    # Ensure input directory exists
    if not os.path.exists(input_dir):
        print(f"Input directory '{input_dir}' does not exist. Creating it...")
        try:
            os.makedirs(input_dir, exist_ok=True)
            print(f"Directory created. Please add your PDB files to {input_dir} and run the script again.")
            return
        except Exception as e:
            print(f"Failed to create directory: {e}")
            return

    # Get full path for output file
    output_file_path = os.path.join(input_dir, output_file)

    # Get list of PDB files
    pdb_files = glob.glob(os.path.join(input_dir, "*.pdb"))
    if not pdb_files:
        print(f"No PDB files found in '{input_dir}'.")
        return

    print(f"Found {len(pdb_files)} PDB files to process.")

    # Process each PDB file
    results = []
    for pdb_file in pdb_files:
        print(f"Processing {os.path.basename(pdb_file)}...")
        result = process_pdb_file(pdb_file, ub_chain, binder_chain, cutoff)
        results.append(result)
        print_result(result)

    # Write results to CSV
    with open(output_file_path, 'w', newline='') as csvfile:
        if results:
            fieldnames = results[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for result in results:
                writer.writerow(result)

            print(f"\nResults written to {output_file_path}")
        else:
            print("No results to write to CSV.")

# Main execution - this will run when the script is executed in Colab
if __name__ == "__main__":
    print("Ubiquitin Interface Calculator for Google Colab")
    print("="*50)

    # Mount Google Drive
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        print("This script is designed to run in Google Colab with Google Drive.")
        sys.exit(1)

    # Run analysis with default parameters
    print(f"Using the following parameters:")
    print(f"  Input directory: {DEFAULT_INPUT_DIR}")
    print(f"  Ubiquitin chain: {DEFAULT_UB_CHAIN}")
    print(f"  Binder chain: {DEFAULT_BINDER_CHAIN}")
    print(f"  Contact cutoff: {DEFAULT_CUTOFF} Å")
    print(f"  Output file: {DEFAULT_OUTPUT_FILE}")
    print("\nTo change these parameters, edit the DEFAULT_* variables at the top of the script.")
    print("="*50)

    run_analysis()

Ubiquitin Interface Calculator for Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Using the following parameters:
  Input directory: /content/drive/MyDrive/BindCraft/UB2-K11/3NOB/Att4-iso/Accepted
  Ubiquitin chain: A
  Binder chain: B
  Contact cutoff: 4.5 Å
  Output file: results.csv

To change these parameters, edit the DEFAULT_* variables at the top of the script.
Found 2 PDB files to process.
Processing 3NOBEK_strand_l81_s451607_mpnn1_model2.pdb...

INTERFACE ANALYSIS FOR K11-LINKED UBIQUITIN BINDER
Input PDB file: 3NOBEK_strand_l81_s451607_mpnn1_model2.pdb

Contact Statistics:
  First Ubiquitin (residues 1-75):
    Atom contacts: 621
    Residue pairs: 32
    Unique Ub1 residues involved: 11
    Unique Binder residues involved: 12
    Average contact distance: 3.83 Å

  Second Ubiquitin (residues 126-201):
    Atom contacts: 763
    Residue pairs: 38
 